In [65]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import requests
import time

def extract_content(url):
    executable_path=r"C:\Users\PC\Desktop\projects\Web_Scraper\chromedriver-win64\chromedriver-win64\chromedriver.exe"
    options = Options()
    service=Service(executable_path=executable_path)
    options.add_argument('--headless')  
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')

    # You can specify the path to chromedriver if it's not in PATH
    driver = webdriver.Chrome(service=service,options=options)

    driver.get(url)
    time.sleep(3)  # Wait for full page to load (can adjust)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

    # Remove unwanted elements
    for tag in soup(['script', 'style', 'nav', 'footer', 'header']):
        tag.decompose()

    text = soup.get_text(separator='\n', strip=True)
    return text




     

In [66]:
from duckduckgo_search import DDGS
def search_url(question,results=3):
    urls=[]
    with DDGS() as ddgs:
        for r in ddgs.text(question,max_results=results):
            urls.append(r['href'])
    return urls

In [68]:
#web search for urls
question=input()
urls=search_url(question)
for i,url in enumerate(urls,0):
    print(f"{i},{url}")


 what is chemistry?


0,https://www.britannica.com/science/chemistry
1,https://en.wikipedia.org/wiki/Chemistry
2,https://chem.libretexts.org/Courses/Furman_University/CHM101:_Chemistry_and_Global_Awareness_(Gordon)/01:_Introduction_to_Chemistry/1.01:_What_is_Chemistry


In [86]:

def summary_prompt(url_content):
    prompt = f"""You are an assistant and your role is to summarize the text clearly, concisely, and include important nuances.
    The summary should include only relevant information for the question.

    Text to summarize:
    {url_content}

    Respond with a Text:
    """
    return prompt
    
    

In [87]:
model="mistral"
def Query_llm(prompt: str, model: str, host: str = "http://localhost:11434"):
    url = f"{host}/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False  # Set to True if you want streaming chunks
    }

    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        return response.text.strip()
    except requests.exceptions.RequestException as e:
        print("Ollama API error:", e)
        return ""

In [88]:
state={}
state["question"]=question
state["context"]={}

In [ ]:
for i ,link in enumerate(urls):
    state["context"][f"link{i}"]=url
    url_content=extract_content(url)
    prompt=summary_prompt(url_content)
    summary=Query_llm(prompt,model)
    state["context"][f"summary{i}"]=summary
print(state["question"])
print(state["context"]["link0"])
print(state["context"]["summary0"])